In [1]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)


(True, 'Tesla P100-PCIE-16GB')

In [2]:
!pip install -U datasets fsspec huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 26.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6.0:
      Successfully uninstalled datasets-3.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.

In [3]:
!pip install transformers datasets gradio scikit-learn


In [4]:
from datasets import load_dataset

# Load the community version of AG News
ds = load_dataset("sh0416/ag_news")

# Preview a sample
print(ds["train"][0])


README.md: 0.00B [00:00, ?B/s]

train.jsonl:   0%|          | 0.00/33.7M [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'label': 3, 'title': 'Wall St. Bears Claw Back Into the Black (Reuters)', 'description': "Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."}


In [5]:
ds = ds.map(lambda x: {"text": x["title"] + " " + x["description"]})


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [8]:
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


2025-07-25 15:47:29.425826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753458449.603073      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753458449.651803      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-news",           # where to save results       # evaluate every epoch
    logging_strategy="epoch",           # log every epoch
    save_strategy="epoch",              # save checkpoint every epoch
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)


In [11]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


In [12]:
tokenized_ds["test"] = tokenized_ds["test"].filter(lambda x: x["label"] in [0, 1, 2, 3])


Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [14]:
import wandb
wandb.login(key="8bae383d5337cade93d49019f3e306f8dff0edfb")
from transformers import Trainer

# Filter out samples with labels outside the valid range [0, 3]
valid_labels = range(model.config.num_labels)
train_dataset_filtered = tokenized_ds["train"].filter(lambda x: x["label"] in valid_labels)
eval_dataset_filtered = tokenized_ds["test"].filter(lambda x: x["label"] in valid_labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_filtered,
    eval_dataset=eval_dataset_filtered,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: c4coding-2005 (c4coding-2005-developers-hub-intern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/tmp/ipykernel_36/2787631393.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
5625,0.148200
11250,0.075700
16875,0.037400


TrainOutput(global_step=16875, training_loss=0.08710803674768519, metrics={'train_runtime': 13502.4603, 'train_samples_per_second': 19.996, 'train_steps_per_second': 1.25, 'total_flos': 7.104126062592e+16, 'train_loss': 0.08710803674768519, 'epoch': 3.0})

In [15]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.1322837620973587, 'eval_accuracy': 0.9754385964912281, 'eval_f1': 0.9754228476820519, 'eval_runtime': 88.9173, 'eval_samples_per_second': 64.104, 'eval_steps_per_second': 4.015, 'epoch': 3.0}


In [16]:
trainer.save_model("./bert-news-final")
tokenizer.save_pretrained("./bert-news-final")


('./bert-news-final/tokenizer_config.json',
 './bert-news-final/special_tokens_map.json',
 './bert-news-final/vocab.txt',
 './bert-news-final/added_tokens.json',
 './bert-news-final/tokenizer.json')

In [17]:
import os
print(os.listdir('/kaggle/working/bert-news-final'))


['model.safetensors', 'vocab.txt', 'tokenizer_config.json', 'config.json', 'special_tokens_map.json', 'tokenizer.json', 'training_args.bin']


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "/kaggle/working/bert-news-final"  # adjust if needed

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [46]:
label_map = {
    0: "World",
    1: "Sci/Tech",
    2: "Sports",
    3: "Business"
}


In [47]:
def predict_topic(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    label_id = torch.argmax(outputs.logits, dim=1).item()
    return label_map[label_id]


In [48]:
headline = "NASA launches new telescope to study distant galaxies"
print(predict_topic(headline))  # Expected: Sci/Tech


Sci/Tech


In [49]:
headline = "Global markets dip as inflation fears grow"
print(predict_topic(headline))  # Expected: Business


Business


In [81]:
!zip -r bert-news-final.zip /kaggle/working/bert-news-final


  adding: kaggle/working/bert-news-final/ (stored 0%)
  adding: kaggle/working/bert-news-final/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/bert-news-final/vocab.txt (deflated 53%)
  adding: kaggle/working/bert-news-final/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/bert-news-final/config.json (deflated 52%)
  adding: kaggle/working/bert-news-final/special_tokens_map.json (deflated 42%)
  adding: kaggle/working/bert-news-final/tokenizer.json (deflated 71%)
  adding: kaggle/working/bert-news-final/training_args.bin (deflated 51%)
